In [37]:
import re
import pandas as pd
from nltk.corpus import stopwords

import numpy as np

import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
#from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [38]:
news = pd.read_excel("C:\\Users\\Kratika\\Downloads\\finance_new.xlsx")
news.head()

,Title,Body,Category,Old MegaCategory,MegaCategory,important
0,JERA to acquire 49% of Reliances power project...,03 Sep 2019 12:27 PM BdST\nUpdated:\n03 Sep 20...,Received Private Funding- Alert me when any of...,Financial,Financial,others
1,CarDekho raises Series C funding of $110 M fro...,Automobile classified platform CarDekho has ra...,Received Private Funding- Alert me when any of...,Financial,Financial,others
2,"IndusInd Bank promoters to pump Rs 2,700 crore...",The merger between IndusInd Bank and the micro...,Received Private Funding- Alert me when any of...,Financial,Financial,others
3,Business Briefs: Mahindra Partners Participate...,"Mahindra Partners, the Incubation, Private Equ...",Received Private Funding- Alert me when any of...,Financial,Financial,others
4,Mitsubishi leads funding for off-grid power fi...,(You can enter multiple email addresses separa...,Received Private Funding- Alert me when any of...,Financial,Financial,others


In [39]:
from nltk.tokenize import word_tokenize 

def remove_stop_words(data):
    stop_words = set(stopwords.words('english'))
    stop_words.union(['january','february','march','april','may','june','july','august','september','october','november','december'])
    stop_words.union(['jan','feb','mar','apr','may','jun','jul','aug','sept','oct','nov','dec'])
    stop_words.union(['monday', 'tuesday', 'wednesday', 'thursday','friday','saturday','sunday'])
    stop_words.union(['am', 'pm'])
    word_tokens = word_tokenize(data) 
    filtered_sentence = []
    
    for w in word_tokens: 
        if w.lower() not in stop_words: 
            filtered_sentence.append(w) 
    
    return " ".join(filtered_sentence)

In [40]:
news['text'] = news.Title + " " + news.Body

In [41]:
# import these modules 
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 
  
print("shareholders :", lemmatizer.lemmatize("shareholder")) 
print("stocks :", lemmatizer.lemmatize("stock")) 
print("companys :", lemmatizer.lemmatize("company")) 
print("trading :", lemmatizer.lemmatize("trade")) 
print("weekly :", lemmatizer.lemmatize("week")) 
print("billion:", lemmatizer.lemmatize("billionaire")) 
print("high :", lemmatizer.lemmatize("higher")) 
print("close :", lemmatizer.lemmatize("closeed")) 
print("finance :", lemmatizer.lemmatize("financial")) 
print("biggest:", lemmatizer.lemmatize("high")) 
print("unit:", lemmatizer.lemmatize("units")) 
print("protect:", lemmatizer.lemmatize("protection")) 
print("organisation:", lemmatizer.lemmatize("organised")) 
print("'company:", lemmatizer.lemmatize("'companies")) 
# a denotes adjective in "pos" 
print("better :", lemmatizer.lemmatize("better", pos ="a"))

shareholders : shareholder
stocks : stock
companys : company
trading : trade
weekly : week
billion: billionaire
high : higher
close : closeed
finance : financial
biggest: high
unit: unit
protect: protection
organisation: organised
'company: 'companies
better : good


In [42]:
 news['text'] = news['text'].apply(lemmatizer.lemmatize)

In [43]:

news['clean_title'] = news['Title'].apply(remove_stop_words)
news['clean_body'] = news['Body'].apply(remove_stop_words)

news['clean_text'] = news['text'].apply(remove_stop_words) 

#remove_stop_words_Body


In [44]:
# ## Split data
print("\nSplitting data")

title_tr, title_te, MegaCategory_tr, MegaCategory_te = train_test_split(news['clean_text'], news.important,test_size =.1)
title_tr, title_de, MegaCategory_tr , MegaCategory_de = train_test_split(title_tr,MegaCategory_tr,test_size =.1)


print("Training: ",len(title_tr))

print("Testing: ",len(title_te))


Splitting data
Training:  199
Testing:  25


In [45]:
from sklearn.preprocessing import LabelEncoder

# # Data Preprocessing
# ## Vectorization of data
# Vectorize the data using Bag of words (BOW)
print("\nVectorizing data")
tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
stop_words = nltk.corpus.stopwords.words("english")
vectorizer = CountVectorizer(tokenizer=tokenizer.tokenize, stop_words=stop_words)

vectorizer.fit(iter(title_tr))
Xtr = vectorizer.transform(iter(title_tr))
Xde = vectorizer.transform(iter(title_de))
Xte = vectorizer.transform(iter(title_te))

encoder = LabelEncoder()
encoder.fit(MegaCategory_tr)
Ytr = encoder.transform(MegaCategory_tr)
Yde = encoder.transform(MegaCategory_de)
Yte = encoder.transform(MegaCategory_te)


Vectorizing data


In [46]:
print(Ytr)

[0 4 0 4 2 1 4 0 0 1 0 5 1 0 2 1 0 4 4 0 0 4 5 2 3 0 4 5 0 0 2 2 5 1 3 0 5
 4 3 4 0 5 0 2 4 4 2 4 0 0 0 2 0 2 3 1 3 4 0 0 2 4 0 4 0 3 5 3 4 0 0 3 4 4
 0 3 0 4 0 4 0 4 3 0 1 2 2 4 1 3 0 0 4 2 4 3 5 3 1 5 3 4 0 3 0 4 2 0 4 1 2
 0 3 2 5 4 4 0 0 3 4 3 0 2 3 2 0 2 0 4 1 4 4 4 4 4 2 1 4 2 0 1 3 4 1 0 0 0
 0 0 0 0 4 3 3 0 4 0 4 3 1 4 4 0 1 1 2 0 1 2 4 2 0 0 3 5 3 3 5 4 3 1 0 4 1
 4 4 4 5 5 5 4 0 0 4 5 4 4 0]


In [47]:
Xtr

<199x3207 sparse matrix of type '<class 'numpy.int64'>'
	with 9449 stored elements in Compressed Sparse Row format>

In [48]:
# # Train Models
# ### Baseline Model
# “stratified”: generates predictions by respecting the training set’s class distribution.
print("\n\nTraining baseline classifier")
dc = DummyClassifier(strategy="stratified")
dc.fit(Xtr, Ytr)

dc_pred = dc.predict(Xde)



Training baseline classifier


In [49]:
print(classification_report(Yde, dc_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(dc_pred == Yde)))

                                           precision    recall  f1-score   support

              Acquisition or Stake Change       0.50      0.50      0.50         8
                    Company Share Related       0.33      0.20      0.25         5
                           Debt Financing       0.50      0.50      0.50         8
Merger, Restructuring or Ownership Change       0.33      0.20      0.25         5
          Research House or Rating Agency       0.20      0.25      0.22         4
                                   others       0.00      0.00      0.00         2

                                 accuracy                           0.38       959
                                macro avg       0.39      0.34      0.35       959
                             weighted avg       0.41      0.37      0.38       959

Accuracy achieved is 0.34782608695652173


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 199, does not match size of target_names, 6
  .format(len(labels), len(target_names))


In [50]:

# ### Decision Tree
print("Training Decision tree")
dt = DecisionTreeClassifier()
dt.fit(Xtr, Ytr)
dt_pred = dt.predict(Xde)
print(classification_report(Yde, dt_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(dt_pred == Yde)))

Training Decision tree
                                           precision    recall  f1-score   support

              Acquisition or Stake Change       0.71      0.62      0.67         8
                    Company Share Related       0.67      0.80      0.73         5
                           Debt Financing       0.71      0.62      0.67         8
Merger, Restructuring or Ownership Change       0.67      0.80      0.73         5
          Research House or Rating Agency       0.33      0.25      0.29         4
                                   others       0.67      1.00      0.80         2

                                 accuracy                           0.67       959
                                macro avg       0.67      0.70      0.67       959
                             weighted avg       0.67      0.67      0.66       959

Accuracy achieved is 0.6521739130434783


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 199, does not match size of target_names, 6
  .format(len(labels), len(target_names))


In [51]:
# ### Random Forest
print("Training Random Forest")
rf = RandomForestClassifier(n_estimators=40)
rf.fit(Xtr, Ytr)
pred = rf.predict(Xde)
print(classification_report(Yde, pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred == Yde)))

Training Random Forest
                                           precision    recall  f1-score   support

              Acquisition or Stake Change       0.50      0.88      0.64         8
                    Company Share Related       0.75      0.60      0.67         5
                           Debt Financing       0.50      0.88      0.64         8
Merger, Restructuring or Ownership Change       0.75      0.60      0.67         5
          Research House or Rating Agency       0.00      0.00      0.00         4
                                   others       1.00      1.00      1.00         2

                                 accuracy                           0.63       959
                                macro avg       0.58      0.66      0.60       959
                             weighted avg       0.55      0.69      0.60       959

Accuracy achieved is 0.6086956521739131


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 199, does not match size of target_names, 6
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [52]:

# ### Multinomial Naive Bayesian
print("Training Multinomial Naive Bayesian")
nb = MultinomialNB()
nb.fit(Xtr, Ytr)
pred_nb = nb.predict(Xde)
print(classification_report(Yde, pred_nb, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred_nb == Yde)))

Training Multinomial Naive Bayesian
                                           precision    recall  f1-score   support

              Acquisition or Stake Change       1.00      1.00      1.00         8
                    Company Share Related       0.67      0.80      0.73         5
                           Debt Financing       1.00      1.00      1.00         8
Merger, Restructuring or Ownership Change       0.67      0.80      0.73         5
          Research House or Rating Agency       1.00      0.75      0.86         4
                                   others       0.50      1.00      0.67         2

                                 accuracy                           0.87       959
                                macro avg       0.78      0.84      0.80       959
                             weighted avg       0.85      0.89      0.86       959

Accuracy achieved is 0.8260869565217391


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 199, does not match size of target_names, 6
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [53]:
# ### Support Vector Classification
print("Training Support Vector Classification")
from sklearn.svm import SVC
svc = SVC()
svc.fit(Xtr, Ytr)
svc_pred = svc.predict(Xde)
print(classification_report(Yde, svc_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(svc_pred == Yde)))

Training Support Vector Classification
                                           precision    recall  f1-score   support

              Acquisition or Stake Change       0.35      1.00      0.52         8
                    Company Share Related       0.00      0.00      0.00         5
                           Debt Financing       0.35      1.00      0.52         8
Merger, Restructuring or Ownership Change       0.00      0.00      0.00         5
          Research House or Rating Agency       0.00      0.00      0.00         4
                                   others       0.00      0.00      0.00         2

                                 accuracy                           0.41       959
                                macro avg       0.10      0.30      0.15       959
                             weighted avg       0.17      0.49      0.25       959

Accuracy achieved is 0.34782608695652173


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 199, does not match size of target_names, 6
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [55]:
# ### Multilayered Perceptron
print("Training Multilayered Perceptron")
mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100, 20), random_state=1, max_iter=400)
mlp.fit(Xtr, Ytr)
mlp_pred = mlp.predict(Xde)
print(classification_report(Yde, mlp_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(mlp_pred == Yde)))

Training Multilayered Perceptron
                                           precision    recall  f1-score   support

              Acquisition or Stake Change       1.00      0.88      0.93         8
                    Company Share Related       0.62      1.00      0.77         5
                           Debt Financing       1.00      0.88      0.93         8
Merger, Restructuring or Ownership Change       0.62      1.00      0.77         5
          Research House or Rating Agency       1.00      0.50      0.67         4
                                   others       0.67      1.00      0.80         2

                                 accuracy                           0.83       959
                                macro avg       0.79      0.82      0.78       959
                             weighted avg       0.85      0.86      0.83       959

Accuracy achieved is 0.782608695652174


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 199, does not match size of target_names, 6
  .format(len(labels), len(target_names))


In [56]:
print('Accuracy achieved is ' + str(np.mean(dc_pred   == Yde)))
print('Accuracy achieved is ' + str(np.mean(dt_pred   == Yde)))
print('Accuracy achieved is ' + str(np.mean(pred_nb   == Yde)))
print('Accuracy achieved is ' + str(np.mean(svc_pred  == Yde)))
print('Accuracy achieved is ' + str(np.mean(mlp_pred  == Yde)))

Accuracy achieved is 0.34782608695652173
Accuracy achieved is 0.6521739130434783
Accuracy achieved is 0.8260869565217391
Accuracy achieved is 0.34782608695652173
Accuracy achieved is 0.782608695652174


In [57]:
# # Final Model: Multilayered Perceptron
# ## Predict test data
print("\n\nPredicting test data using Multilayered Perceptron")
pred_final = mlp.predict(Xte)
print(classification_report(Yte, pred_final, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(accuracy_score(Yte,pred_final)))



Predicting test data using Multilayered Perceptron
                                           precision    recall  f1-score   support

              Acquisition or Stake Change       0.82      1.00      0.90         9
                    Company Share Related       1.00      0.86      0.92         7
                           Debt Financing       0.82      1.00      0.90         9
Merger, Restructuring or Ownership Change       1.00      0.86      0.92         7
          Research House or Rating Agency       0.67      1.00      0.80         2
                                   others       1.00      0.50      0.67         2

                                 accuracy                           0.88      1114
                                macro avg       0.79      0.80      0.78      1114
                             weighted avg       0.86      0.90      0.87      1114

Accuracy achieved is 0.84


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [58]:
output = {"title":vectorizer.inverse_transform(Xte), "predicted":encoder.inverse_transform(pred_final),"actual": encoder.inverse_transform(Yte)}

In [59]:
#df = pd.DataFrame(output, columns=["title","predicted","actual"])
#df.to_csv("C:\\Users\\Kratika\\Downloads\\News333_title_predication_1.csv")

In [60]:
pred_final = mlp.predict(Xte)
print('Accuracy achieved is ' + str(np.mean(pred_final == Yte)))
output = {"text":vectorizer.inverse_transform(Xte), "predicted":encoder.inverse_transform(pred_final),"actual": encoder.inverse_transform(Yte)}
df = pd.DataFrame(output, columns=["text","predicted","actual"])


Accuracy achieved is 0.84


In [61]:
df = pd.DataFrame(output, columns=["text","predicted","actual"])
df.to_csv("C:\\Users\\Kratika\\Downloads\\old.csv")

In [62]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer


#GET VECTOR COUNT
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(news.clean_text)

#SAVE WORD VECTOR
pickle.dump(count_vect.vocabulary_, open("count_vector.pkl","wb"))

In [63]:
X_train_counts

<247x3529 sparse matrix of type '<class 'numpy.int64'>'
	with 11052 stored elements in Compressed Sparse Row format>

In [64]:
from sklearn.feature_extraction.text import TfidfTransformer

#TRANSFORM WORD VECTOR TO TF IDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#SAVE TF-IDF
pickle.dump(tfidf_transformer, open("tfidf.pkl","wb"))


In [65]:
X_train_tfidf

<247x3529 sparse matrix of type '<class 'numpy.float64'>'
	with 11052 stored elements in Compressed Sparse Row format>

In [66]:
from sklearn.neural_network import MLPClassifier

#mlp =       MLPClassifier(solver='adam',  alpha=1e-5, hidden_layer_sizes=(100, 20), random_state=1, max_iter=400)
#clf_neural = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)
clf_neural = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)
#clf_neural = MLPClassifier(hidden_layer_sizes=(100, ), activation=’relu’, solver=’adam’, alpha=0.0001, batch_size=’auto’, learning_rate=’constant’, learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10)
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, news.important, test_size=0.25, random_state=42)

clf_neural.fit(X_train, y_train)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(15,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='sgd', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [67]:
predicted = clf_neural.predict(X_test)
print('Accuracy achieved is ' + str(np.mean(predicted == y_test)))


Accuracy achieved is 0.24193548387096775


In [68]:
print(classification_report(y_test, predicted))

                                           precision    recall  f1-score   support

              Acquisition or Stake Change       0.24      1.00      0.39        15
                    Company Share Related       0.00      0.00      0.00         8
                           Debt Financing       0.00      0.00      0.00         6
Merger, Restructuring or Ownership Change       0.00      0.00      0.00        10
          Research House or Rating Agency       0.00      0.00      0.00        17
                                   others       0.00      0.00      0.00         6

                                 accuracy                           0.24        62
                                macro avg       0.04      0.17      0.06        62
                             weighted avg       0.06      0.24      0.09        62



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [88]:
mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100,), activation='relu', random_state=1, max_iter=400)

mlp.fit(X_train, y_train)
mlp_pred = mlp.predict(X_test)  
print(classification_report(y_test, mlp_pred))
print('Accuracy achieved is ' + str(np.mean(mlp_pred == y_test)))

                                           precision    recall  f1-score   support

              Acquisition or Stake Change       0.78      0.93      0.85        15
                    Company Share Related       1.00      0.62      0.77         8
                           Debt Financing       0.75      1.00      0.86         6
Merger, Restructuring or Ownership Change       0.89      0.80      0.84        10
          Research House or Rating Agency       0.85      1.00      0.92        17
                                   others       0.50      0.17      0.25         6

                                 accuracy                           0.82        62
                                macro avg       0.79      0.75      0.75        62
                             weighted avg       0.81      0.82      0.80        62

Accuracy achieved is 0.8225806451612904
